# 🚀 Premier Backtest avec NautilusTrader

Ce notebook vous guide pas à pas pour créer votre premier backtest avec NautilusTrader.

## Objectifs :
1. ✅ Télécharger des données historiques (Yahoo Finance)
2. ✅ Les convertir au format NautilusTrader
3. ✅ Créer une stratégie MACD simple
4. ✅ Lancer un backtest
5. ✅ Analyser les résultats

---

## 📦 1. Imports et Configuration

In [1]:
import warnings
warnings.filterwarnings('ignore')

from decimal import Decimal
from pathlib import Path
import pandas as pd
import yfinance as yf

# NautilusTrader imports
from nautilus_trader.backtest.node import BacktestNode, BacktestRunConfig, BacktestVenueConfig, BacktestDataConfig, BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig, LoggingConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model.data import Bar, BarType
from nautilus_trader.model.identifiers import InstrumentId, Symbol, Venue
from nautilus_trader.model.instruments import Equity
from nautilus_trader.model.objects import Price, Quantity, Money
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider

print("✅ Imports réussis !")

✅ Imports réussis !


## 📊 2. Téléchargement des Données

On va télécharger 2 ans de données journalières pour Apple (AAPL)

In [2]:
# Paramètres
SYMBOL = "AAPL"
START_DATE = "2022-01-01"
END_DATE = "2024-01-01"

# Téléchargement
print(f"📥 Téléchargement des données {SYMBOL} de {START_DATE} à {END_DATE}...")
df = yf.download(SYMBOL, start=START_DATE, end=END_DATE, progress=False)

print(f"✅ {len(df)} barres téléchargées")
print(f"\nAperçu des données:")
df.head()

📥 Téléchargement des données AAPL de 2022-01-01 à 2024-01-01...
✅ 501 barres téléchargées

Aperçu des données:


Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2022-01-03  178.443115  179.296076  174.227395  174.345039  104487900
2022-01-04  176.178406  179.354917  175.609770  179.050994   99310400
2022-01-05  171.492065  176.639180  171.217554  176.090157   94537600
2022-01-06  168.629303  171.864636  168.276357  169.315582   96904000
2022-01-07  168.795929  170.727323  167.678270  169.501820   86709100

## 🔄 3. Conversion au Format NautilusTrader

NautilusTrader utilise un catalogue Parquet pour stocker les données de manière optimisée.

In [3]:
# Créer le dossier catalog s'il n'existe pas
CATALOG_PATH = Path("../data/catalog")
CATALOG_PATH.mkdir(parents=True, exist_ok=True)

# Initialiser le catalogue
catalog = ParquetDataCatalog(str(CATALOG_PATH))
print(f"📁 Catalogue créé dans: {CATALOG_PATH}")

📁 Catalogue créé dans: ..\data\catalog


In [7]:
# Créer l'instrument
instrument_id = InstrumentId(symbol=Symbol(SYMBOL), venue=Venue("NASDAQ"))

# Créer un instrument Equity simple
from nautilus_trader.model.currencies import USD  # Import ajouté

instrument = Equity(
    instrument_id=instrument_id,
    raw_symbol=Symbol(SYMBOL),
    currency=USD,                           # ✅ Objet USD
    price_precision=2,
    price_increment=Price.from_str("0.01"),
    lot_size=Quantity.from_int(1),          # ✅ Seul ce paramètre est nécessaire
    ts_event=0,
    ts_init=0,
)

print(f"🎯 Instrument créé: {instrument_id}")

🎯 Instrument créé: AAPL.NASDAQ


In [8]:
# Définir le type de barre
bar_type = BarType.from_str(f"{instrument_id}-1-DAY-LAST-EXTERNAL")
print(f"📊 Type de barre: {bar_type}")

# Convertir les données avec BarDataWrangler
wrangler = BarDataWrangler(bar_type=bar_type, instrument=instrument)

# Préparer le DataFrame au format attendu
df_processed = df.copy()
df_processed.columns = [col.lower() for col in df_processed.columns]

# Convertir en barres Nautilus
bars = wrangler.process(df_processed)
print(f"✅ {len(bars)} barres converties au format Nautilus")

📊 Type de barre: AAPL.NASDAQ-1-DAY-LAST-EXTERNAL


AttributeError: 'tuple' object has no attribute 'lower'

In [ ]:
# Écrire dans le catalogue
catalog.write_data([instrument])
catalog.write_data(bars)

print("✅ Données écrites dans le catalogue")
print(f"\n📋 Instruments dans le catalogue:")
print(catalog.instruments())

## 🎯 4. Création de la Stratégie MACD

Stratégie simple basée sur le MACD:
- **Achat** quand MACD croise au-dessus de la ligne de signal
- **Vente** quand MACD croise en-dessous de la ligne de signal

In [5]:
%%writefile ../strategies/macd_strategy.py

from decimal import Decimal
from nautilus_trader.config import StrategyConfig
from nautilus_trader.core.data import Data
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model.data import Bar
from nautilus_trader.model.enums import OrderSide, TimeInForce
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.orders import MarketOrder
from nautilus_trader.trading.strategy import Strategy


class MACDStrategyConfig(StrategyConfig):
    """Configuration pour la stratégie MACD"""
    instrument_id: str
    bar_type: str
    fast_period: int = 12
    slow_period: int = 26
    signal_period: int = 9
    trade_size: Decimal = Decimal("100")


class MACDStrategy(Strategy):
    """
    Stratégie MACD simple:
    - Achat quand MACD > Signal
    - Vente quand MACD < Signal
    """

    def __init__(self, config: MACDStrategyConfig):
        super().__init__(config)
        
        # Configuration
        self.instrument_id = InstrumentId.from_str(config.instrument_id)
        self.bar_type = config.bar_type
        self.trade_size = config.trade_size
        
        # Indicateur MACD
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period,
            slow_period=config.slow_period,
            signal_period=config.signal_period,
        )
        
        # État
        self.position_opened = False

    def on_start(self):
        """Actions au démarrage de la stratégie"""
        self.subscribe_bars(self.bar_type)
        self.log.info(f"Stratégie démarrée pour {self.instrument_id}")

    def on_bar(self, bar: Bar):
        """Appelé à chaque nouvelle barre"""
        # Mettre à jour l'indicateur
        self.macd.handle_bar(bar)
        
        # Attendre que l'indicateur soit initialisé
        if not self.macd.initialized:
            return
        
        # Récupérer les valeurs
        macd_value = self.macd.value
        signal_value = self.macd.signal
        
        # Logique de trading
        if macd_value > signal_value and not self.position_opened:
            # Signal d'achat
            self.buy()
            
        elif macd_value < signal_value and self.position_opened:
            # Signal de vente
            self.sell()

    def buy(self):
        """Ouvrir une position longue"""
        order = self.order_factory.market(
            instrument_id=self.instrument_id,
            order_side=OrderSide.BUY,
            quantity=self.instrument.make_qty(self.trade_size),
            time_in_force=TimeInForce.GTC,
        )
        self.submit_order(order)
        self.position_opened = True
        self.log.info(f"📈 ACHAT: MACD={self.macd.value:.4f}, Signal={self.macd.signal:.4f}")

    def sell(self):
        """Fermer la position"""
        order = self.order_factory.market(
            instrument_id=self.instrument_id,
            order_side=OrderSide.SELL,
            quantity=self.instrument.make_qty(self.trade_size),
            time_in_force=TimeInForce.GTC,
        )
        self.submit_order(order)
        self.position_opened = False
        self.log.info(f"📉 VENTE: MACD={self.macd.value:.4f}, Signal={self.macd.signal:.4f}")

    def on_stop(self):
        """Actions à l'arrêt de la stratégie"""
        # Fermer toutes les positions ouvertes
        self.close_all_positions(self.instrument_id)
        self.log.info("Stratégie arrêtée")


Writing ../strategies/macd_strategy.py


## ⚙️ 5. Configuration du Backtest

In [6]:
# Importer la stratégie
import sys
sys.path.append('../strategies')
from macd_strategy import MACDStrategy, MACDStrategyConfig

print("✅ Stratégie importée")

ModuleNotFoundError: No module named 'nautilus_trader.indicators.macd'

In [ ]:
# Configuration de la venue (exchange simulé)
venue_config = BacktestVenueConfig(
    name="NASDAQ",
    oms_type="HEDGING",
    account_type="CASH",  # Compte cash pour actions
    base_currency="USD",
    starting_balances=["100000 USD"],  # Capital de départ
)

print("✅ Configuration venue créée")
print(f"  - Venue: {venue_config.name}")
print(f"  - Capital de départ: 100,000 USD")

In [ ]:
# Récupérer l'instrument du catalogue
instruments = catalog.instruments()
instrument = instruments[0]

# Dates pour le backtest
start = dt_to_unix_nanos(pd.Timestamp(START_DATE, tz='UTC'))
end = dt_to_unix_nanos(pd.Timestamp(END_DATE, tz='UTC'))

print(f"📅 Période du backtest:")
print(f"  - Début: {START_DATE}")
print(f"  - Fin: {END_DATE}")

In [ ]:
# Configuration des données
data_config = BacktestDataConfig(
    catalog_path=str(CATALOG_PATH),
    data_cls=Bar,
    instrument_id=instrument.id,
    start_time=start,
    end_time=end,
)

print("✅ Configuration données créée")

In [ ]:
# Configuration de la stratégie
strategy_config = ImportableStrategyConfig(
    strategy_path="macd_strategy:MACDStrategy",
    config_path="macd_strategy:MACDStrategyConfig",
    config={
        "instrument_id": str(instrument.id),
        "bar_type": str(bar_type),
        "fast_period": 12,
        "slow_period": 26,
        "signal_period": 9,
        "trade_size": Decimal("10"),  # 10 actions par trade
    },
)

print("✅ Configuration stratégie créée")
print(f"  - MACD rapide: 12")
print(f"  - MACD lent: 26")
print(f"  - Signal: 9")
print(f"  - Taille de trade: 10 actions")

In [ ]:
# Configuration complète du backtest
config = BacktestRunConfig(
    engine=BacktestEngineConfig(
        strategies=[strategy_config],
        logging=LoggingConfig(log_level="ERROR"),  # ERROR pour éviter trop de logs
    ),
    data=[data_config],
    venues=[venue_config],
)

print("✅ Configuration backtest complète créée")

## 🚀 6. Lancement du Backtest

⚠️ **Note**: Le backtest peut prendre quelques secondes à quelques minutes selon la quantité de données.

In [ ]:
# Créer et lancer le backtest node
print("🚀 Lancement du backtest...\n")

node = BacktestNode(configs=[config])
results = node.run()

print("\n✅ Backtest terminé !")

## 📊 7. Analyse des Résultats

In [ ]:
# Récupérer le moteur pour l'analyse
engine = node.get_engine(config.id)

print("📈 RÉSUMÉ DU BACKTEST")
print("=" * 60)

In [ ]:
# Rapport des ordres
print("\n📋 RAPPORT DES ORDRES:")
print("=" * 60)
engine.trader.generate_order_fills_report()

In [ ]:
# Rapport du compte
print("\n💰 RAPPORT DU COMPTE:")
print("=" * 60)
engine.trader.generate_account_report(Venue("NASDAQ"))

In [ ]:
# Rapport des positions
print("\n📊 RAPPORT DES POSITIONS:")
print("=" * 60)
engine.trader.generate_positions_report()

In [ ]:
# Statistiques de performance détaillées
from nautilus_trader.model.identifiers import Venue

# Récupérer le compte
account = engine.cache.account_for_venue(Venue("NASDAQ"))

if account:
    print("\n💵 PERFORMANCE FINALE:")
    print("=" * 60)
    print(f"Capital de départ: 100,000.00 USD")
    print(f"Capital final: {account.balance_total().as_double():,.2f} USD")
    
    pnl = account.balance_total().as_double() - 100000.0
    pnl_pct = (pnl / 100000.0) * 100
    
    print(f"P&L: {pnl:+,.2f} USD ({pnl_pct:+.2f}%)")
    
    if pnl > 0:
        print("\n🎉 Stratégie profitable !")
    else:
        print("\n⚠️ Stratégie non profitable - ajustez les paramètres")
else:
    print("❌ Impossible de récupérer les informations du compte")

## 🎯 8. Prochaines Étapes

Maintenant que vous avez réussi votre premier backtest, vous pouvez :

1. **Optimiser les paramètres** : Testez différentes valeurs de MACD (fast_period, slow_period, signal_period)
2. **Ajouter des filtres** : RSI, bandes de Bollinger, volume, etc.
3. **Multi-symboles** : Backtester sur plusieurs actions simultanément
4. **Améliorer la stratégie** : Stop-loss, take-profit, trailing stop
5. **Visualisation** : Créer des graphiques de la performance

---

### 💡 Conseils :
- Commencez simple et ajoutez de la complexité progressivement
- Testez toujours sur des données out-of-sample
- Documentez vos résultats
- N'oubliez pas : les performances passées ne garantissent pas les résultats futurs !